# GroupfMRI14a - Automated SRActFlow using ANN-inspired representations
# Testing compositional RITL - train model on held-out 60 tasks. Test model on held-out 4 task contexts.
#### Using ActFlow

#### Takuya Ito
#### 06/03/2019

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools_group_rsa as tools_group
import nibabel as nib
import EmpiricalSRActFlow_ANN_RSA_v2 as esr
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"
import pandas as pd
import time


In [2]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# Load real motor response data

In [3]:
tools_group = reload(tools_group)
nResponses = 2
data_task_rh = np.zeros((len(glasser2),nResponses,len(subjNums)))
data_task_lh = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task_rh[:,:,scount] = np.real(tools_group.loadMotorResponses(subj,hand='Right'))
    data_task_lh[:,:,scount] = np.real(tools_group.loadMotorResponses(subj,hand='Left'))
    scount += 1

tools_group_rsa.py:95: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,0] = data[:,2]
tools_group_rsa.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,1] = data[:,3]
tools_group_rsa.py:98: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,0] = data[:,0] #lmid
tools_group_rsa.py:99: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,1] = data[:,1] #lind


#### Identify target vertices

In [4]:
# Set indices for layer-by-layer vertices
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))

target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)

# Define functions for motor response decodings

#### Load in FC mappings

In [5]:
tools_group = reload(tools_group)
inputtypes = ['color','ori','pitch','constant']
inputkeys = ['RED','VERTICAL','HIGH','CONSTANT']
fc_input2hidden = {}
eig_input2hidden = {}
i = 0
for inputtype in inputtypes:
    fc_input2hidden[inputkeys[i]], eig_input2hidden[inputkeys[i]] = tools_group.loadGroupActFlowFC(inputtype,pc_space=True)
    i += 1
    
# Load rules to hidden FC mappings
fc_logic2hidden, eig_logic2hidden = tools_group.loadGroupActFlowFC('Logic',pc_space=True)
fc_sensory2hidden, eig_sensory2hidden = tools_group.loadGroupActFlowFC('Sensory',pc_space=True)
fc_motor2hidden, eig_motor2hidden = tools_group.loadGroupActFlowFC('Motor',pc_space=True)

# Load hidden to motor resp mappings
fc_hidden2motorresp, eig_hidden2motorresp = tools_group.loadGroupActFlowFC('hidden2out',pc_space=True)

#### Generate trials to simulate empirical brain computational models

#### Define wrapper function for SRActFlow

In [467]:
esr = reload(esr)
filename_training='/projects3/SRActFlow/data/results/GroupfMRI/RSA/GroupfMRI14a_Trainset_60PracticeTasks_50stims_v1.csv'
filename_testing='/projects3/SRActFlow/data/results/GroupfMRI/RSA/GroupfMRI14a_Testset_4NovelTasks_50stims_v1.csv'
# Create training set (60 stimului per task)
esr.constructTasksForRITLGeneralization(n_stims=12,filename_training=filename_training,filename_testing=filename_testing)
# Create testing set (900 stimuli per task)
# esr.constructTasksForRITLGeneralization(n_stims=20,filename_training='tmp.txt',filename_testing=filename_testing)
# This ensures practice and novel tasks have the same number of trials

trial_practice_metadata = pd.read_csv(filename_training)
trial_novel_metadata = pd.read_csv(filename_testing)
full_metadata = trial_practice_metadata.append(trial_novel_metadata)

In [468]:
print np.sum((trial_novel_metadata.motorResp.values=='LMID'))
print np.sum((trial_novel_metadata.motorResp.values=='LIND'))
print np.sum((trial_novel_metadata.motorResp.values=='RMID'))
print np.sum((trial_novel_metadata.motorResp.values=='RIND'))

96
96
96
96


In [469]:
print np.sum((trial_novel_metadata.motorResp.values=='LMID'))
print np.sum((trial_novel_metadata.motorResp.values=='LIND'))
print np.sum((trial_novel_metadata.motorResp.values=='RMID'))
print np.sum((trial_novel_metadata.motorResp.values=='RIND'))

96
96
96
96


In [470]:
tmp = ['LMID','LIND','RMID','RIND']
print 'RMID', np.sum((trial_practice_metadata.motorResp.values=='RMID') | (trial_practice_metadata.motorRule.values=='RMID'))
print 'RMID', np.sum((trial_practice_metadata.motorResp.values=='RMID') | (trial_practice_metadata.motorRule.values=='RIND'))

print 'RIND', np.sum((trial_practice_metadata.motorResp.values=='RIND') | (trial_practice_metadata.motorRule.values=='RMID'))
print 'RIND', np.sum((trial_practice_metadata.motorResp.values=='RIND') | (trial_practice_metadata.motorRule.values=='RIND'))

print 'LMID', np.sum((trial_practice_metadata.motorResp.values=='LMID') | (trial_practice_metadata.motorRule.values=='LMID'))
print 'LMID', np.sum((trial_practice_metadata.motorResp.values=='LMID') | (trial_practice_metadata.motorRule.values=='LIND'))

print 'LIND', np.sum((trial_practice_metadata.motorResp.values=='LIND') | (trial_practice_metadata.motorRule.values=='LMID'))
print 'LIND', np.sum((trial_practice_metadata.motorResp.values=='LIND') | (trial_practice_metadata.motorRule.values=='LIND'))

RMID 144
RMID 144
RIND 144
RIND 144
LMID 144
LMID 144
LIND 144
LIND 144


In [ ]:
esr = reload(esr)
def subjSRActFlow_PCFC((subj,trial_practiced_metadata,trial_novel_metadata)):
    print('Subject ' + subj + '... Simulating ' + str(len(trial_practiced_metadata)) + ' Practiced Trials | ' + str(len(trial_novel_metadata)) + ' Novel Trials')
    obj = esr.EmpiricalActFlow(subj)
    # Input
    obj.fc_input2hidden = fc_input2hidden
    obj.eig_input2hidden = eig_input2hidden
    # Rules
    obj.fc_logic2hidden = fc_logic2hidden
    obj.eig_logic2hidden = eig_logic2hidden
    obj.fc_sensory2hidden = fc_sensory2hidden
    obj.eig_sensory2hidden = eig_sensory2hidden
    obj.fc_motor2hidden = fc_motor2hidden
    obj.eig_motor2hidden = eig_motor2hidden
    # hidden 2 motor
    obj.fc_hidden2motorresp = fc_hidden2motorresp
    obj.eig_hidden2motorresp = eig_hidden2motorresp
    
    # Generate actflow predictions for practiced tasks
    obj.extractAllActivations(trial_practiced_metadata)
    actflow_practiced = obj.generateActFlowPredictions_PCFC(thresh=0,verbose=False)
    del obj
    
    #### Run for novel tasks
    obj = esr.EmpiricalActFlow(subj)
    # Input
    obj.fc_input2hidden = fc_input2hidden
    obj.eig_input2hidden = eig_input2hidden
    # Rules
    obj.fc_logic2hidden = fc_logic2hidden
    obj.eig_logic2hidden = eig_logic2hidden
    obj.fc_sensory2hidden = fc_sensory2hidden
    obj.eig_sensory2hidden = eig_sensory2hidden
    obj.fc_motor2hidden = fc_motor2hidden
    obj.eig_motor2hidden = eig_motor2hidden
    # hidden 2 motor
    obj.fc_hidden2motorresp = fc_hidden2motorresp
    obj.eig_hidden2motorresp = eig_hidden2motorresp
    # Generate actflow predictions for novel tasks
    obj.extractAllActivations(trial_novel_metadata)
    actflow_novel = obj.generateActFlowPredictions_PCFC(thresh=0,verbose=False)
    del obj
    
    return actflow_practiced, actflow_novel

global fc_input2hidden
global fc_logic2hidden
global fc_sensory2hidden
global fc_motor2hidden

inputs = []
for i in range(len(subjNums)):
    inputs.append((subjNums[i],trial_practice_metadata,trial_novel_metadata))

timestart = time.time()
pool = mp.Pool(processes=16)
results = pool.map_async(subjSRActFlow_PCFC,inputs).get()
pool.close()
pool.join()
timeend = time.time()
print "time elapsed:", timeend-timestart

actflow_predictions_practiced = np.zeros((len(subjNums),len(target_ind),4))
actflow_predictions_novel = np.zeros((len(subjNums),len(target_ind),4))
scount = 0
for result in results:
    actflow_predictions_practiced[scount,:,:] = result[0]
    actflow_predictions_novel[scount,:,:] = result[1]
    scount += 1



Subject 013... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 017... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 023... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 027... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 031... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 034... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 038... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 041... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 024... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 014... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 032... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 018... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 035... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 028... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 039... Simulating 384 Practiced Trials | 384 Novel Trials
Subject 04

#### Compute average activity for each response, for each subject

In [460]:
nResponses = 2
actflow_rh_practice = np.zeros((len(glasser2),nResponses,len(subjNums)))
actflow_lh_practice = np.zeros((len(glasser2),nResponses,len(subjNums)))
actflow_rh_novel = np.zeros((len(glasser2),nResponses,len(subjNums)))
actflow_lh_novel = np.zeros((len(glasser2),nResponses,len(subjNums)))
for scount in range(len(subjNums)):
    # RMID
    actflow_rh_practice[target_ind,0,scount] = actflow_predictions_practiced[scount,:,2]
    # RIND
    actflow_rh_practice[target_ind,1,scount] = actflow_predictions_practiced[scount,:,3]
    # LMID
    actflow_lh_practice[target_ind,0,scount] = actflow_predictions_practiced[scount,:,0]
    # LIND
    actflow_lh_practice[target_ind,1,scount] = actflow_predictions_practiced[scount,:,1]
    
    # RMID
    actflow_rh_novel[target_ind,0,scount] = actflow_predictions_novel[scount,:,2]
    # RIND
    actflow_rh_novel[target_ind,1,scount] = actflow_predictions_novel[scount,:,3]
    # LMID
    actflow_lh_novel[target_ind,0,scount] = actflow_predictions_novel[scount,:,0]
    # LIND
    actflow_lh_novel[target_ind,1,scount] = actflow_predictions_novel[scount,:,1]
    
thresh = 0
if thresh!=None:
    actflow_rh_practice = np.multiply(actflow_rh_practice,actflow_rh_practice>thresh)
    actflow_lh_practice = np.multiply(actflow_lh_practice,actflow_lh_practice>thresh)
    actflow_rh_novel = np.multiply(actflow_rh_novel,actflow_rh_novel>thresh)
    actflow_lh_novel = np.multiply(actflow_lh_novel,actflow_lh_novel>thresh)

    



# Run across subject decoding on right-hand motor responses

#### Run recurrent actflow on motor cortex

In [461]:
recurrence = 0
h5f = h5py.File('/projects3/SRActFlow/data/results/GroupfMRI/RecurrentMotorFC/group_weightsrh.h5','r')
smnweights = h5f['weights'][:].copy()


rois = np.asarray([7,8,52])
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

data_orig = data_task_rh[roi_ind,:,:,].copy()
noveldata = actflow_rh_novel[roi_ind,:,:].copy()
practicedata = actflow_rh_practice[roi_ind,:,:].copy()
# data_orig = stats.zscore(data_orig,axis=0).copy()
noveldata = stats.zscore(noveldata,axis=0).copy()
practicedata = stats.zscore(practicedata,axis=0).copy()

for i in range(recurrence):
    for scount in range(len(subjNums)):
        for cond in range(data_task_rh.shape[1]):
            noveldata[:,cond,scount] = noveldata[:,cond,scount] + np.matmul(noveldata[:,cond,scount],smnweights[1:,:])
            practicedata[:,cond,scount] = practicedata[:,cond,scount] + np.matmul(practicedata[:,cond,scount],smnweights[1:,:])
            

#### Run decoding

In [462]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1



distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch = tools_group.compositionalActflowDecodings(data_orig,noveldata,
                                                        
                                                                                        practicedata,effects=True,
                                                          ncvs=ncvs, featsel=True, nproc=nproc)

#### Compute statistics

In [463]:
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print 'Activity flow accuracy =', statistics_rh[0,0]
print 'p =', statistics_rh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



Activity flow accuracy = 0.208333333333
p = 1.0
Matched spatial correlation: -0.0164061615059
Mismatched spatial correlation: 0.0164061615059


# Run across subject decoding on left-hand motor responses

#### Run recurrent actflow on motor cortex

In [464]:
recurrence = 0
h5f = h5py.File('/projects3/SRActFlow/data/results/GroupfMRI/RecurrentMotorFC/group_weightslh.h5','r')
smnweights = h5f['weights'][:].copy()


rois = np.asarray([187,188,232])
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
data_orig = data_task_lh[roi_ind,:,:,].copy()
noveldata = actflow_lh_novel[roi_ind,:,:].copy()
practicedata = actflow_lh_practice[roi_ind,:,:].copy()
# data_orig = stats.zscore(data_orig,axis=0).copy()
noveldata = stats.zscore(noveldata,axis=0).copy()
practicedata = stats.zscore(practicedata,axis=0).copy()

for i in range(recurrence):
    for scount in range(len(subjNums)):
        for cond in range(data_task_lh.shape[1]):
            noveldata[:,cond,scount] = noveldata[:,cond,scount] + np.matmul(noveldata[:,cond,scount],smnweights[1:,:])
            practicedata[:,cond,scount] = practicedata[:,cond,scount] + np.matmul(practicedata[:,cond,scount],smnweights[1:,:])
            

#### Run decoding

In [465]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1


distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch = tools_group.compositionalActflowDecodings(data_orig,noveldata,
                                                          practicedata,effects=True, featsel=True,
                                                          ncvs=ncvs, nproc=nproc)
    

#### Compute statistics

In [466]:
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print 'Activity flow accuracy =', statistics_lh[0,0]
print 'p =', statistics_lh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



Activity flow accuracy = 0.395833333333
p = 0.997622381257
Matched spatial correlation: -0.00641757214721
Mismatched spatial correlation: 0.00641757214721


___